<a href="https://colab.research.google.com/github/Estebanarci2/Marketing-analitica3/blob/main/C)MODELOS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>